<a href="https://colab.research.google.com/github/amphyxs/know-graphs-cw/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -r requirements.txt

In [17]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer
from typing import Optional, List
import re

# Инициализиация онтологии

In [18]:
g = Graph()

g.parse('ontology.rdf')
ONTOLOGY_IRI = 'http://www.semanticweb.org/user/ontologies/2024/9/music-knowledge'

def ref(resource: str) -> URIRef:
  return URIRef(f'{ONTOLOGY_IRI}#{resource}')

# Python-объекты для классов в онтологии

In [19]:
class OntologyClass:
  class_name: str
  _class_uri_ref: Optional[URIRef] = None
  uri_ref: URIRef

  @property
  def class_uri_ref(self) -> URIRef:
    if not self._class_uri_ref:
      self._class_uri_ref = ref(self.class_name)

    return self._class_uri_ref

  def __init__(self, name: str, **kwargs: dict) -> None:
    self.uri_ref = ref(self._prepare_name_for_ref(name))
    g.add((self.uri_ref, RDF.type, self.class_uri_ref))

  def _prepare_name_for_ref(self, name: str) -> str:
    name = name.replace(' ', '_')

    return re.sub(r'[^A-Za-z0-9_]', '', name)

In [20]:
class Artist(OntologyClass):
  class_name = 'Исполнитель'

class Song(OntologyClass):
  owns_object_property = ref('владеть')
  class_name = 'Композиция'

  def __init__(self, name: str, owner: Artist, **kwargs: dict) -> None:
    super().__init__(name, **kwargs)
    g.add((owner.uri_ref, self.owns_object_property, self.uri_ref))

# Получение инстансов для классов из API

In [21]:
import requests

API_URL = "https://itunes.apple.com/search"

def fetch_musical_bands(limit: int = 10) -> List[str]:
    url = API_URL
    params = {
        "term": "band",        # General search term to find bands
        "media": "music",
        "entity": "musicArtist",
        "limit": limit,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    return [artist['artistName'] for artist in results]

def fetch_songs_for_band(band_name: str, limit: int = 10) -> List[str]:
    url = API_URL
    params = {
        "term": band_name,
        "media": "music",
        "entity": "song",
        "limit": limit,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    return [song['trackName'] for song in results]

bands = fetch_musical_bands()
for band in bands:
    songs = fetch_songs_for_band(band)
    artist_class_instance = Artist(band)
    for idx, song in enumerate(songs, 1):
        song_class_instance = Song(song, artist_class_instance)


In [22]:
g.serialize(destination='ontology-from-py.rdf')

<Graph identifier=Na9218b28364a48e895d13fdba1c90895 (<class 'rdflib.graph.Graph'>)>